## Importar paquetes

In [7]:
import mapbox_tilesets
print("📦 Módulo:", mapbox_tilesets)
print("📂 Ubicación del paquete:", mapbox_tilesets.__file__)

📦 Módulo: <module 'mapbox_tilesets' from '/Users/sergio/Projects/RedwoodForest/.venv/lib/python3.6/site-packages/mapbox_tilesets/__init__.py'>
📂 Ubicación del paquete: /Users/sergio/Projects/RedwoodForest/.venv/lib/python3.6/site-packages/mapbox_tilesets/__init__.py


In [6]:
import os, json, re, unicodedata
from pathlib import Path
from mapbox_tilesets import Tilesets


ImportError: cannot import name 'Tilesets'

In [4]:
!which python

/Users/sergio/Projects/RedwoodForest/.venv/bin/python


In [ ]:
# Sustituye esto con tu token real de Mapbox
MAPBOX_ACCESS_TOKEN = "pk.XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# Inicializa la conexión
tilesets = Tilesets(access_token=MAPBOX_ACCESS_TOKEN)

# Intenta listar los tilesets existentes
try:
    response = tilesets.list()
    print(f"✅ Conexión exitosa. Tilesets disponibles: {len(response)}")
except Exception as e:
    print("❌ Error de conexión:", e)


## Variables

In [ ]:
MAPBOX_USER  = "sergio"                # <- tu usuario Mapbox
DATE_TAG     = "07072025"              # AAAAMMDD
SRC_DIR      = Path("data/FR_Layers_Valid")  # carpeta con 26 GeoJSON

## Token

In [ ]:
# 2) TOKEN (cárgalo del entorno o ponlo directo — NO subas el token a Git)
token = os.getenv("MAPBOX_ACCESS_TOKEN", "pk.XXXXXXXXXXXXXXXXXXXXXXXX")
assert token.startswith("pk."), "⚠️  MAPBOX_ACCESS_TOKEN no definido"


## Cliente

In [ ]:
ts = Tilesets()
print("Tilesets CLI version:", ts.__version__)
print("Carpeta fuente:", SRC_DIR.resolve())

## Funciones auxiliares

In [ ]:
def slugify(name: str) -> str:
    """Pasa a minúsculas, reemplaza espacios y quita símbolos no válidos"""
    name = name.lower()
    name = re.sub(r"[^\w_]+", "_", name)   # sólo a-z 0-9 _
    name = re.sub(r"__+", "_", name)       # colapsa doble guión bajo
    return name.strip("_")[:24]            # corta a 24 caract. (queda margen)

def make_recipe(source_id: str, layer_name: str, minzoom=6, maxzoom=14):
    return {
        "version": 1,
        "layers": {
            layer_name: {
                "source": f"mapbox://tileset-source/{MAPBOX_USER}/{source_id}",
                "minzoom": minzoom,
                "maxzoom": maxzoom
            }
        }
    }

Bucle de carga masiva

In [ ]:
for geojson in sorted(SRC_DIR.glob("*.geojson")):
    basename     = slugify(geojson.stem)
    source_id    = f"{MAPBOX_USER}_src_{DATE_TAG}_{basename}"
    tileset_id   = f"{MAPBOX_USER}_rff_{DATE_TAG}_{basename}"

    print(f"\n⬆️  {geojson.name} → {tileset_id}")

    # 1. SUBIR / REEMPLAZAR SOURCE
    rsp = ts.upload_source(MAPBOX_USER, source_id, str(geojson))
    print("   • Source upload:", rsp.status_code)

    # 2. CREAR O ACTUALIZAR TILESET
    recipe = make_recipe(source_id, basename)

    # ¿Existe ya?
    existing = [t["id"] for t in ts.list().json()["tilesets"]]
    if tileset_id in existing:
        rsp = ts.update(tileset_id, name=basename, recipe=recipe)
        print("   • Tileset update:", rsp.status_code)
    else:
        rsp = ts.create(tileset_id, name=basename, recipe=recipe)
        print("   • Tileset create:", rsp.status_code)

    # 3. PUBLICAR
    rsp = ts.publish(tileset_id)
    job_id = rsp.json()["jobId"]
    print("   • Publish job:", job_id)

print("\n✅ Todos los jobs enviados. Revisa progreso en Mapbox Studio → Tilesets.")